In [ ]:
#go to kaggle and get ur own api and load the api in
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"canbemodified","key":"73818061a3d1aa1d697f25c211b0ad27"}'}

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Downloading of IAM dataset from Kaggle

In [ ]:
!kaggle datasets download -d bigkizd/iam-dataset

100% 3.73G/3.73G [01:10<00:00, 29.4MB/s]
100% 3.73G/3.73G [01:10<00:00, 57.2MB/s]


In [ ]:
#this takes around 1-2 mins
!unzip iam-dataset.zip

In [ ]:
!pip install keras_tqdm

In [ ]:
!pip install pyspellchecker

In [ ]:

import numpy as np
import cv2
import os
import pandas as pd
import string
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional, GRU
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
#from keras_tqdm import TQDMNotebookCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from spellchecker import SpellChecker

from google.colab.patches import cv2_imshow

In [ ]:
with open('iam/IAM/words.txt') as f:
    contents = f.readlines()

lines = [line.strip() for line in contents] 
#lines[18]
#the file name start from 18


In [ ]:
#lines[18].split(" ")

['a01-000u-00-00', 'ok', '154', '408', '768', '27', '51', 'AT', 'A']

# Encoding of image labels

In [ ]:
max_label_len = 0

char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz" 

print(char_list, len(char_list))

def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, chara in enumerate(txt):
        dig_lst.append(char_list.index(chara))
        
    return dig_lst

In [ ]:
train_images = []
train_labels = []
train_input_length = []
train_label_length = []
train_original_text = []

valid_images = []
valid_labels = []
valid_input_length = []
valid_label_length = []
valid_original_text = []

test_images = []
test_labels = []
test_input_length = []
test_label_length = []
test_original_text = []

inputs_length = []
labels_length = []
test_length = []


images = []
labels = []
test = []



# Processing of image


In [ ]:
def process_image(img):
    """
    Converts image to shape (32, 128, 1) & normalize
    """
    w, h = img.shape
    
    
    # bilateral filter
    img = cv2.bilateralFilter(img, 9, 75, 75)
    
    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape
    
    img = img.astype('float32')
    
    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape
    
    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape
        
    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)
    
    img = cv2.subtract(255, img)
    
    img = np.expand_dims(img, axis=2)
    
    # Normalize 
    img = img / 255
    
    return img

# Loading of data


In [ ]:
for index, line in enumerate(lines[18:]):
    splits = line.split(' ')
    status = splits[1]
    
    if status == 'ok':
        word_id = splits[0]
        word = "".join(splits[8:])
        
        splits_id = word_id.split('-')
        filepath = 'iam/IAM/words/{}/{}-{}/{}.png'.format(splits_id[0], 
                                                  splits_id[0], 
                                                  splits_id[1], 
                                                  word_id)
        
        # process image
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        try:
            img = process_image(img)
        except:
            continue
            
        # process label
        try:
            label = encode_to_labels(word)
        except:
            continue
        
        if index % 5 == 0:
            valid_images.append(img)
            valid_labels.append(label)
            valid_input_length.append(31)
            valid_label_length.append(len(word))
            valid_original_text.append(word)
        elif index % 8 == 0:
            test_images.append(img)
            test_labels.append(label)
            test_input_length.append(31)
            test_label_length.append(len(word))
            test_original_text.append(word)
        else:
            train_images.append(img)
            train_labels.append(label)
            train_input_length.append(31)
            train_label_length.append(len(word))
            train_original_text.append(word)
        
        if len(word) > max_label_len:
            max_label_len = len(word)
    


# Padding of text labels into max label length

In [ ]:
train_padded_label = pad_sequences(train_labels, 
                             maxlen=max_label_len, 
                             padding='post',
                             value=len(char_list))

valid_padded_label = pad_sequences(valid_labels, 
                             maxlen=max_label_len, 
                             padding='post',
                             value=len(char_list))

test_padded_label = pad_sequences(test_labels, 
                             maxlen=max_label_len, 
                             padding='post',
                             value=len(char_list))


In [ ]:
train_padded_label.shape, valid_padded_label.shape, test_padded_label.shape

((67510, 21), (19292, 21), (9652, 21))

# Model parameters
 - Convolution layers followed by max pooling layers
 - For batch normalisation layers, batchnorm is added before activation

In [ ]:
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# pooling layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same' )(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = None, padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
activated_batchnorm_5 = relu(batch_norm_5, alpha=0.0, max_value=None, threshold=0)
 
conv_6 = Conv2D(512, (3,3), activation = None, padding='same')(activated_batchnorm_5)
batch_norm_6 = BatchNormalization()(conv_6)
activated_batchnorm_6 = relu(batch_norm_6, alpha=0.0, max_value=None, threshold=0)
pool_6 = MaxPool2D(pool_size=(2, 1))(activated_batchnorm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
pool_7 = MaxPool2D(pool_size=(1, 1))(conv_7)

conv_8 = Conv2D(1024, (1,1), activation = 'relu')(pool_7)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)
 
# bidirectional LSTM layers with units=512 and dropout of 0.2
blstm_1 = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)
act_model.summary()

# Model with no batch normalization

In [ ]:
#No batch normalization
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# pooling layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same' )(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# pooling layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)

 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(conv_5)
pool_6 = MaxPool2D(pool_size=(2, 1))(conv_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
pool_7 = MaxPool2D(pool_size=(1, 1))(conv_7)

conv_8 = Conv2D(1024, (1,1), activation = 'relu')(pool_7)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)
 
# bidirectional LSTM layers with units=512
blstm_1 = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(512, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)
act_model.summary()

# Model with one layer of GRU instead of Bidirection LSTM

In [ ]:
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# pooling layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same' )(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# pooling layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = None, padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
activated_batchnorm_5 = relu(batch_norm_5, alpha=0.0, max_value=None, threshold=0)
 
conv_6 = Conv2D(512, (3,3), activation = None, padding='same')(activated_batchnorm_5)
batch_norm_6 = BatchNormalization()(conv_6)
activated_batchnorm_6 = relu(batch_norm_6, alpha=0.0, max_value=None, threshold=0)
pool_6 = MaxPool2D(pool_size=(2, 1))(activated_batchnorm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
pool_7 = MaxPool2D(pool_size=(1, 1))(conv_7)

conv_8 = Conv2D(1024, (1,1), activation = 'relu')(pool_7)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)
#use one layer of GRU to predict
gru = GRU(512,
            activation='tanh',
            recurrent_activation='sigmoid',
            dropout=0.2,
            recurrent_dropout = 0,
            unroll = False,
            use_bias = True,
            reset_after = True,
            return_sequences=True,
            )(squeezed)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(gru)

# model to be used at test time
act_model = Model(inputs, outputs)
# act_model.summary()

# Model with 2 layers of GRU 

In [ ]:
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same' )(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = None, padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
activated_batchnorm_5 = relu(batch_norm_5, alpha=0.0, max_value=None, threshold=0)
 
conv_6 = Conv2D(512, (3,3), activation = None, padding='same')(activated_batchnorm_5)
batch_norm_6 = BatchNormalization()(conv_6)
activated_batchnorm_6 = relu(batch_norm_6, alpha=0.0, max_value=None, threshold=0)
pool_6 = MaxPool2D(pool_size=(2, 1))(activated_batchnorm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
pool_7 = MaxPool2D(pool_size=(1, 1))(conv_7)

conv_8 = Conv2D(1024, (1,1), activation = 'relu')(pool_7)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)
#use two layer of GRU with bidirection to predict
gru_1 = Bidirectional(GRU(512,
            activation='tanh',
            recurrent_activation='sigmoid',
            dropout=0.2,
            recurrent_dropout = 0,
            unroll = False,
            use_bias = True,
            reset_after = True,
            return_sequences=True,
            ))(squeezed)
gru_2 = Bidirectional(GRU(512,
            activation='tanh',
            recurrent_activation='sigmoid',
            dropout=0.2,
            recurrent_dropout = 0,
            unroll = False,
            use_bias = True,
            reset_after = True,
            return_sequences=True,
            ))(gru_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(gru_2)

# model to be used at test time
act_model = Model(inputs, outputs)
act_model.summary()
act_model.save("content/drive/My drive/CS3244_Project/Models/GRU_bidirectional_two_layer")

# Hyperparameters used for model training 

In [ ]:
batch_size = 128
epochs = 20
e = str(epochs)
optimizer_name = "Adam"

In [ ]:
train_images = np.asarray(train_images)
train_input_length = np.asarray(train_input_length)
train_label_length = np.asarray(train_label_length)

valid_images = np.asarray(valid_images)
valid_input_length = np.asarray(valid_input_length)
valid_label_length = np.asarray(valid_label_length)

test_images = np.asarray(test_images)
test_input_length = np.asarray(test_input_length)
test_label_length = np.asarray(test_label_length)

# Training the model using CTC decode and CTC loss

In [ ]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = "Adam", metrics=['accuracy'])
filepath="{}o-{}r-{}e-{}t-{}v.hdf5".format("Adam",
                                          str(10000),
                                          str(epochs),
                                          str(train_images.shape[0]),
                                          str(valid_images.shape[0]))

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

history = model.fit(x=[train_images, train_padded_label, train_input_length, train_label_length],
                    y=np.zeros(len(train_images)),
                    batch_size = batch_size, 
                    epochs = 20, 
                    validation_data = ([valid_images, valid_padded_label, valid_input_length, valid_label_length], [np.zeros(len(valid_images))]),
                    verbose=1,
                    callbacks = callbacks_list)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

# Plotting training accuarcy vs validation accuracy

In [ ]:

plt.plot(epochs, loss, 'b')
plt.plot(epochs, val_loss, 'r')
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plotting training cost vs validation cost

In [ ]:

plt.plot(epochs, acc, 'b')
plt.plot(epochs, val_acc, 'r')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Using levenshtein distance




In [1]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )

    return (matrix[size_x - 1, size_y - 1])

# Using levenshtein distance to evaluate model performance

In [ ]:
act_model.load_weights(filepath)
prediction = act_model.predict(test_images)
 
# use CTC decoder
decoded = K.ctc_decode(prediction, 
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]
out = K.get_value(decoded)


distance = 0
total_ratio = 0
# see the results
for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]
    distance = levenshtein(letters, test_original_text[i])
    total_len = len(letters) + len(test_original_text[i])

    ratio = (total_len - distance) / total_len

    total_ratio += ratio


average_ratio = total_ratio / len(out)

average_ratio


0.9535318336037378

# Using accuracy metrics to evaluate model performance


In [ ]:
act_model.load_weights(filepath)
prediction = act_model.predict(test_images)
 
# use CTC decoder
decoded = K.ctc_decode(prediction, 
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]
out = K.get_value(decoded)

count = 0 

for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]
    if letters == test_original_text[i]:
      count+=1

"The accuracy of the model is {}%".format(count/test_padded_label.shape[0])


# using levenshtein based spellchecker to reduce errors

In [ ]:
no_spellcheck_errors = 0
spellcheck_errors = 0
wrong_corrections = 0
spell = SpellChecker()

for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]

    
    if letters != valid_original_text[i]:
      no_spellcheck_errors += 1

    corrected =  spell.correction(letters)

    if  corrected != valid_original_text[i]:
      spellcheck_errors += 1

    if corrected != valid_original_text[i] and letters == valid_original_text:
      wrong_corrections += 1

print("No Spellcheck Errors:" + str(no_spellcheck_errors))
print("Spellcheck Errors:" + str(spellcheck_errors))
print("Wrong corrections:" + str(wrong_corrections))
    

# Error analysis

In [ ]:
for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]

    
    if letters != valid_original_text[i]:
      print("Output:" + letters + "  | | Correct:" + valid_original_text[i])
      plt.imshow(valid_images[i].squeeze())
      plt.show()

In [ ]:
prediction = act_model.predict(train_images)

# use CTC decoder
decoded = K.ctc_decode(prediction, 
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]
out = K.get_value(decoded)

for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]

    
    if letters != train_original_text[i]:
      print("Output:" + letters + "  | | Correct:" + train_original_text[i])
      plt.imshow(train_images[i].squeeze())
      plt.show()